<img src='graphics/text_cleaning.png'>

<img src='graphics/spacer.png'>

<center><font style="font-size:40px;">Cleaning Text from Web Scraping Indeed </font></center>
<center>Prepared and coded by Ben P. Meredith, Ed.D.</center>


When we were last together, we began developing a program to web scrape job announcements from Indeed. In our program, we saved vital information from the job announcements to a Pandas DataFrame. 

We were also left with a few tasks to code prior to today's discussion. As you may recall, I tasked you to do the following:

>1. Find and remove duplicate job announcements
>1. Identify if a table already exists for a search term
    - if it does exist, add new entries to the bottom of the table
    - find and remove duplicate job announcements

If you took the opportunity to work on the code for this program, you realized that the second task (Identify if a table already exists for a search term) required you to do a bit of investigation on your own. I hope that you took advantage of this opportunity and went out to StackOverFlow.com for your research. This task was not one that we covered prior to your challenge, but we will cover it in this notebook. 

There was a third task that we needed to do in order to make our data more valuable. If you took the opportunity to look at the data as it was pulled from Indeed, you will have noticed that it is far from clean. In fact, it is downright dirty with HTML marks and code. Later in this notebook, we will work together on cleaning that data so it is easier for us to process and thus work with it. This work will be our introduction to the data science sub-field called **Natural Language Processing**, which we will discuss below. But first, let's go over the assigned tasks and look at solutions. 

# Loading Fresh Libraries and Data
## Loading Libraries

In [1]:
#Import our needed libraries

import urllib
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
from tqdm import tqdm
from datetime import datetime

## Loading our Pull Functions

In [2]:
def job_data_pull(url):
    page = requests.get(url)# go to the page noted by the url
    page_contents = BeautifulSoup(page.content, 'lxml')#extract the contents of the page
    
    #only getting the tags for organic job postings and not the ones that are sponsored
    tags = page_contents.find_all('div', {'data-tn-component' : "organicJob"})
    
    #getting the list of companies that have the organic job posting tags
    companies = [x.span.text for x in tags]
    
    #extracting the features like the company name, complete link, date, etc.
    attributes = [x.h2.a.attrs for x in tags]
    dates = [x.find_all('span', {'class':'date'}) for x in tags]
    
    # update attributes dictionaries with company name and date posted
    [attributes[i].update({'company': companies[i].strip()}) for i, x in enumerate(attributes)]
    [attributes[i].update({'date posted': dates[i][0].text.strip()}) for i, x in enumerate(attributes)]
    return attributes

## Identify if a Data Table with our Scrapes Already Exists using a Function

In [3]:
# Determine if a file exists within a pathway
def find_file(pathway):#pathway is the path to the file
    from pathlib import Path
    pathway = Path(pathway)# convert the pathway to an actual path from a string
    if pathway.exists():#Determine if the file exists
        return 1 #1 = file exists
    else: 
        return 0 # 0 = file does not exist

In [4]:
#Initialize the data_log by discovering if it exists. If it does, load it. Otherwise, form one. 
def initialize_data_log(job_title):
    import pandas as pd
    job_title = job_title.replace(' ', '_')
    data_file = ('data/'+job_title+'_job_search.csv')
    answer = find_file(data_file)
    if answer == 1:
        df = pd.read_csv(data_file, index_col=0)
        df = df.drop(['level_0'], axis=1, errors='ignore')#Drops level_0 column that keeps showing up 
    else:
        df = pd.DataFrame(columns=('job_id', 'title', 'company', 'url', 'text', 'pull_date'))
        df.to_csv(data_file)
    return df


## Our Basic Scraping Program so We Can Grab Some Data for this Notebook Work

In [5]:
#Ask the user what job description they are interested in searching for and where
job_title = input('What job description are you interested in searching? ')
location = input('What is your zip code?' )

#establish a DF to store the data if one does not exist
#if a search term data already exists, use it.

df = initialize_data_log(job_title)


#Establishes the variables we will need for the Indeed Search URL
getVars = {'q' : job_title, 'l' : location, 'fromage' : 'last', 'sort' : 'date'}

#Assembles the Indeed Search URL from the attributes above
url = ('https://www.indeed.com/jobs?' + urllib.parse.urlencode(getVars))

#Using our uniquely assembled URL, we run the subroutine to get the job data from the function we defined above.
answer = job_data_pull(url)

starting_length = len(df)

# Using the data gathered by our function, we are assigning information to our DataFrame AND we are 
# getting the information we need to retrieve the job description text. 
for index, a in tqdm(enumerate(answer)):
    df.loc[starting_length + index, 'url'] = a['id'].replace('jl_', 'https://www.indeed.com/viewjob?jk=')
    df.loc[starting_length + index, 'title'] = a['title']
    df.loc[starting_length + index, 'company'] = a['company']
    df.loc[starting_length + index, 'job_id'] = a['id']
    df.loc[starting_length + index, 'pull_date'] = datetime.date(datetime.now())

# Using the URLs we generated in line 20, we use them in this FOR loop to retrieve the job description text. 
for index, url in tqdm(enumerate(df.url)):
    try:
        page = requests.get(url)
        content = BeautifulSoup(page.content, "html.parser")
        job_text = content.find('div', class_="jobsearch-jobDescriptionText")
        df.loc[index, 'text'] = str(job_text)
    except KeyError:
        df.loc[index, 'text'] = str(job_text)

job_title = job_title.replace(' ', '_')
df.to_csv('data/'+job_title+'_job_search.csv')
        
df

What job description are you interested in searching? data scientist
What is your zip code?98101


14it [00:00, 610.56it/s]
42it [00:35,  1.17it/s]


,job_id,title,company,url,text,pull_date,clean_text
0,jl_ef4f45eee0d53000,"Engineering Manager, Ads Machine Learning",Pinterest,https://www.indeed.com/viewjob?jk=ef4f45eee0d5...,"<div class=""jobsearch-jobDescriptionText"" id=""...",2020-05-25,<p>About Pinterest:<p>\n<p>\nMillions of peopl...
1,jl_b0fe1a91ffb0e24d,Applied Scientist,Amazon.com Services LLC,https://www.indeed.com/viewjob?jk=b0fe1a91ffb0...,"<div class=""jobsearch-jobDescriptionText"" id=""...",2020-05-25,"- MS or PhD degree in computer science, opera..."
2,jl_900d5e7e57d3225e,Data Scientist,Amazon.com Services LLC,https://www.indeed.com/viewjob?jk=900d5e7e57d3...,"<div class=""jobsearch-jobDescriptionText"" id=""...",2020-05-25,- PhD or equivalent Master's Degree plus 4+ y...
3,jl_a0de3ccca1e027a6,Sr Director of Data and Analytics,Equiscript,https://www.indeed.com/viewjob?jk=a0de3ccca1e0...,"<div class=""jobsearch-jobDescriptionText"" id=""...",2020-05-25,"At Equiscript, we improve access to healthcare..."
4,jl_5d0204fe56e8daee,"ES Tech, Machine Learning Engineer",Amazon.com Services LLC,https://www.indeed.com/viewjob?jk=5d0204fe56e8...,"<div class=""jobsearch-jobDescriptionText"" id=""...",2020-05-25,- Programming experience with at least one mo...
5,jl_cbd17d6de801946f,"Artificial Intelligence Architect, Manager - A...",Deloitte,https://www.indeed.com/viewjob?jk=cbd17d6de801...,"<div class=""jobsearch-jobDescriptionText"" id=""...",2020-05-25,Strategy &amp; Analytics-Analytics and Cogniti...
6,jl_81b253498377e948,Conversational Artificial Intelligence Archite...,Deloitte,https://www.indeed.com/viewjob?jk=81b253498377...,"<div class=""jobsearch-jobDescriptionText"" id=""...",2020-05-25,Strategy &amp; Analytics-Analytics and Cogniti...
7,jl_9e3fac3bc07e40a7,"Computer Vision Architect, Manager - Applied A...",Deloitte,https://www.indeed.com/viewjob?jk=9e3fac3bc07e...,None,2020-05-25,Strategy &amp; Analytics-Analytics and Cogniti...
8,jl_9fb36189ab1f631e,Software Development Engineer – Alexa AI- Seattle,Amazon.com Services LLC,https://www.indeed.com/viewjob?jk=9fb36189ab1f...,"<div class=""jobsearch-jobDescriptionText"" id=""...",2020-05-25,- 2+ years of non-internship professional sof...
9,jl_f317c915a56c7653,Conversational Artificial Intelligence Archite...,Deloitte,https://www.indeed.com/viewjob?jk=f317c915a56c...,"<div class=""jobsearch-jobDescriptionText"" id=""...",2020-05-25,Strategy &amp; Analytics-Analytics and Cogniti...


In [6]:
df.shape

(42, 7)

# Cleaning Text

## The .replace( ) Command

When it comes to cleaning text, the most basic and most often used command is the `.replace()` command. As you can safely assume, the `.replace()` command replaces text or special characters (user defined) with a user defined word, letters, or special character. 

The syntax of the `.replace()` command is simple:

> text = text.replace('what you want to replace', 'what you want it replaced with')

Let's look at the following example. 

In [7]:
# Our test string

test_text = 'The jolly merchant of Verona'

For our `test_text`, we want to replace the word 'jolly' with the word 'happy'. Below is how we do that. 

In [8]:
test_text = test_text.replace('jolly', 'happy')

print(test_text)

The happy merchant of Verona


In our first example, we replaced an entire word. In the next example, we want to replace the punctuation. 

In [9]:
test_text = 'You are kidding.'

In [10]:
test_text = test_text.replace('.', '!')

print(test_text)

You are kidding!


## Exercise 1: Using the .replace( ) to clean text

Now that we have covered the `.replace()` command, you can use it to develop a text cleaning function. 

>1. In the next block, pull the text from several rows in your DataFrame to examine them
>1. Develop a text cleaning function using the `.replace()` command
>1. When you have a function that works for you, run every entry in the ['text'] column in the DataFrame through the function and save the results in a new column titled 'clean_text'.
>1. Finally, print out a sample of 5 entries from the 'clean_text' column alone to make sure your function is working. 

<font color='red'>**HINT**:</font> Think carefully about what you pull and what you replace text with. If you consider it carefully rather than jumping right into the task, you can make your text human readable and human pretty. 

In [11]:
# Write a FOR loop that prints out the text from the first four rows of our DataFrame. 
# Make a separator between each full text entry - try "print('\n','*'*72)" to see what this does. 

for iterator in range(0, 4):
    print(df.loc[iterator, 'text'], '\n')
    print('*'*72)

<div class="jobsearch-jobDescriptionText" id="jobDescriptionText"><div><p><b>About Pinterest:</b></p>
<p>
Millions of people across the world come to Pinterest to find new ideas every day. It’s where they get inspiration, dream about new possibilities and plan for what matters most. Our mission is to help those people find their inspiration and create a life they love. As a Pinterest employee, you’ll be challenged to take on work that upholds this mission and pushes Pinterest forward. You’ll grow as a person and a leader in your field, all the while helping users make their lives better in the positive corner of the internet.</p>
<p>
Before scaling their spend, advertisers want to know that Pinterest is performing for them. Come lead the team that interfaces with advertisers, collecting their sales data and matching it to our internal user databases, and build models and systems to connect the dots between the aspirations of pinners and the products offered by our partners.</p>
<p><b>


In [12]:
# Write a function using the .replace() command that cleans the text. 
# Hint: It may take several lines of code.

def text_cleaner(text):
    text = str(text)
    text = text.replace('<div class="jobsearch-jobDescriptionText" id="jobDescriptionText">', '')
    text = text.replace('<b>', '')
    text = text.replace('</b>', '')
    text = text.replace('<br/>', '')
    text = text.replace('<div>', '')
    text = text.replace('</div>', '')
    text = text.replace('<li>', ' - ')
    text = text.replace('</li>', '')
    text = text.replace('<ul>', '')
    text = text.replace('</ul>', '')
    text = text.replace('</p>', '<p>')
    return text

In [13]:
# Write a FOR loop that cleans and prints out the text from the first four rows of our DataFrame.
# Place a separator between the text. 


for iterator in range(0, 4):
    print(text_cleaner(df.loc[iterator, 'text']), '\n\n', '*'*72, '\n\n')

<p>About Pinterest:<p>
<p>
Millions of people across the world come to Pinterest to find new ideas every day. It’s where they get inspiration, dream about new possibilities and plan for what matters most. Our mission is to help those people find their inspiration and create a life they love. As a Pinterest employee, you’ll be challenged to take on work that upholds this mission and pushes Pinterest forward. You’ll grow as a person and a leader in your field, all the while helping users make their lives better in the positive corner of the internet.<p>
<p>
Before scaling their spend, advertisers want to know that Pinterest is performing for them. Come lead the team that interfaces with advertisers, collecting their sales data and matching it to our internal user databases, and build models and systems to connect the dots between the aspirations of pinners and the products offered by our partners.<p>
<p>
What you’ll do<p>

 - Build and improve the machine learning models for ads measurem

Finally, write a loop that 
>1. pulls the raw text from each row in the DataFrame,
>1. cleans the raw text,
>1. places the clean text in a column called 'clean_text' in the appropriate row for each text, and
>1. print out a sample of five entries from the DataFrame to verify our work.

In [14]:
for index, text in enumerate(df['text']):
    df.loc[index, 'clean_text'] = text_cleaner(text)
    
df.sample(5)

,job_id,title,company,url,text,pull_date,clean_text
37,jl_9fb36189ab1f631e,Software Development Engineer – Alexa AI- Seattle,Amazon.com Services LLC,https://www.indeed.com/viewjob?jk=9fb36189ab1f...,"<div class=""jobsearch-jobDescriptionText"" id=""...",2020-05-26,- 2+ years of non-internship professional sof...
13,jl_ae385acd8d2ea925,Senior Data & Applied Scientist,Microsoft,https://www.indeed.com/viewjob?jk=ae385acd8d2e...,"<div class=""jobsearch-jobDescriptionText"" id=""...",2020-05-25,Analytics team in Azure Hardware Systems &amp;...
30,jl_900d5e7e57d3225e,Data Scientist,Amazon.com Services LLC,https://www.indeed.com/viewjob?jk=900d5e7e57d3...,"<div class=""jobsearch-jobDescriptionText"" id=""...",2020-05-26,- PhD or equivalent Master's Degree plus 4+ y...
14,jl_ef4f45eee0d53000,"Engineering Manager, Ads Machine Learning",Pinterest,https://www.indeed.com/viewjob?jk=ef4f45eee0d5...,"<div class=""jobsearch-jobDescriptionText"" id=""...",2020-05-25,<p>About Pinterest:<p>\n<p>\nMillions of peopl...
41,jl_a03acf446a8a2c6b,Senior Data Scientist,Amazon.com Services LLC,https://www.indeed.com/viewjob?jk=a03acf446a8a...,"<div class=""jobsearch-jobDescriptionText"" id=""...",2020-05-26,"Bachelor or Master's degree in Statistics, App..."


## Tokenizing Words 

"Tokenizing" is the splitting of a text entry into smaller parts, which we call 'tokens'. Tokens can be individual words, individual sentences, or individual paragraphs depending upon what unit of measure you want to use. For our purposes right now, we want to tokenize our clean_text by words.

There are several techniques to splitting up a string into words. We will look at two methods. 

### Method 1: The .split( ) Command

The `.split()` command is not specific to tokenizing words, but we can use it to split a string into words. The `.split()` command can be used to split a string almost anyway that you want it to split; how it splits a string depends upon how you want it to split. 

The syntax of the `.split()` command is simple:
> text = text.split('what spaces, letters or character by which you want the string split')

In the function below, notice this syntax in line 3. 

But what are we doing in line 2?

In [15]:
# Function to using the split command to tokenize a string by words. 

def tokenize_by_words2(text):
    text = text.replace('\n', ' ')
    text = text.split(' ')
    return text

### Method 2: Using nltk to Tokenize by Words

The second method to tokenize a string by words is to use the nltk library's `word_tokenize()` function. 

"NLTK is a leading platform for building Python programs to work with human language data. It provides easy-to-use interfaces to over 50 corpora and lexical resources such as WordNet, along with a suite of text processing libraries for classification, tokenization, stemming, tagging, parsing, and semantic reasoning, and wrappers for industrial-strength NLP libraries." [https://www.nltk.org/]

In the next function, you will note that
>1. in line 4, I added an `import` statement import the needed function out of the nltk library
>1. in lines 5 & 6, I added the `str()` command to the tokenized text so that it saves as a tokenized string

In [16]:
#Tokenize a document's text given as a string to the word level and returning a list of tokenized words

def tokenize_by_words(text):
    from nltk.tokenize import word_tokenize
    token_text = str(word_tokenize(text))
    return str(token_text)

## Cleaning White Space


### Clean Leading and Trailing White Spaces

In [17]:
test_string = ' The   Red fox   Jumped  over The  Lazy dog. '

In [18]:
phrase = test_string.strip()
    
print(phrase)

The   Red fox   Jumped  over The  Lazy dog.


### Clean White Space using .replace( )

In [19]:
test_string = test_string.replace('   ', ' ')
test_string = test_string.replace('  ', ' ')
test_string.strip()

'The Red fox Jumped over The Lazy dog.'

## Normalizing Text

### Make all words lower case using .lower( )

In [20]:
test_string.lower()

' the red fox jumped over the lazy dog. '

### Make all words upper case using .upper( )

In [21]:
test_string.upper()

' THE RED FOX JUMPED OVER THE LAZY DOG. '

### Make the First Letter in Each Word Uppercase using .title( )

In [22]:
test_string.title()

' The Red Fox Jumped Over The Lazy Dog. '

## Exercise

As you look at the output from the line above, you see that we have spaces and at the end of the line. Each word in the string is capitalized. We want to correct these two issues. 

In the next block, using what you have learned in all of our lessons, write a function that would take any string and return a string that is correct with capitalization and non-capitalized words.

After writing the function, run the function on the list of strings in the second block. Did you get it correct?

In [23]:
# Write a function to correct the text capitalization
# The function should capitalize the first letter only in each string.

def correct_captialization(strings_list):
    new_string = []
    for string in strings_list:
        string = string.lower()
        new_string.append(string[0].upper()+string[1:])
    return new_string

In [24]:
# Use the following strings list to test your function

strings_list = ['THE large Box cOntained Her gift.', 'Jack DoeS Not LiKe Getting Water.']

print(correct_captialization(strings_list))


['The large box contained her gift.', 'Jack does not like getting water.']


Let's now save our DataFrame for future use.

In [26]:
job_title = job_title.replace(' ', '_')
df.to_csv('data/'+job_title+'_job_search.csv')